# Exploring Algebras and Coalgebras

This notebooks explores some examples of F-Algebras and F-Coalgebras. It uses the package
`MLStyle.jl` in order to provide some macros that eases the process of writing in a functional programming style.

In [11]:
using Pkg
Pkg.activate(".")
using MLStyle

  Activating project at `~/Documents/GitHub/CTViz_Workshop/Notebooks`


# Example 1

In this example we implement the functor $F_C X := 1 + C \times X$, which has as fixed point the data type of
lists containing values of type `C`, i.e. $List_C = 1 + C \times List_C$.

Thus, note that varying `C` we get the list of different types.

### Catamorphism


We can perceive that there is a general construction. Which is defined as

```Haskell
cata alg = alg . fmap (cata alg) . unFix
```

A theorem by Lambek states that for the initial F-Algebra, which is `(List{c}, fix)` in our case,
there is an inverse function to `fix`, which we call `unfix`. While
`fix(x::ListF{c, List{c}})::List{c}`, the inverse take a vaue of `List{c}` and returns
a value of type `ListF{c,List{c}}`. Let's implement this.

In [36]:
cata(alg::Function, x) = alg(fmap(y -> cata(alg, y), unfix(x)))
cata(alg::Function) = x -> cata(alg,x) 

cata (generic function with 2 methods)

### Implementing the Initial Algebra

In [41]:
struct NilF end
struct ConsF{c,a}
    _1::c
    _2::a
end
ListF{c,a} = Union{NilF, ConsF{c,a}}
fmap(f::Function, x::ConsF) = ConsF(x._1,f(x._2))
fmap(f::Function, x::NilF)  = NilF()


# data List c = Nil | Cons c (List c) ----- In Haskell
struct Nil end
struct Cons{c}
    _1::c
    _2::Union{Nil, Cons{c}}
end
List{c} = Union{Nil, Cons{c}}

fix(::NilF)  = Nil()
fix(x::ConsF)= Cons(x._1,x._2)

unfix(::Nil) = NilF()
unfix(x::Cons)= ConsF(x._1,x._2)

unfix (generic function with 2 methods)

Auxliar function to turn common Vector from Julia [] into List 

In [42]:
function vectolist(x::Vector)
    if x == []
        return Nil()
    end
    (n,ns) = Iterators.peel(x)
    ns = collect(ns)
    return Cons(n,vectolist(ns))
end

listtovec(x::Nil) = []
listtovec(x::Cons{T}) where T = convert(Vector{T},vcat(x._1,listtovec(x._2)))

vectolist([1.0,2.0,3.0])

Cons{Float64}(1.0, Cons{Float64}(2.0, Cons{Float64}(3.0, Nil())))

### F-Algebra

In [43]:
algSum(::NilF)   = 0
algSum(x::ConsF{Int,Int}) = x._1 + x._2;

algProd(::NilF)   = 1
algProd(x::ConsF{Int,Int}) = x._1 * x._2;



algStr(::NilF)   = ""
algStr(x::ConsF{String,String}) = x._1 * x._2; # Note that the functor is actually F{Int} Int
# Thus, ConsF{String,String} is actually F{String} String
# Therefore, algStr : F{String} String -> String

In [44]:
@show cata(algSum, vectolist([1,2,3,4]))
@show cata(algProd, vectolist([1,2,3,4]));
@show cata(algStr, vectolist(["a","b","c"]));

cata(algSum, vectolist([1, 2, 3, 4])) = 10
cata(algProd, vectolist([1, 2, 3, 4])) = 24
cata(algStr, vectolist(["a", "b", "c"])) = "abc"


### Coalgebras and Anamorphism
The co-algebra is the dual of an F-Algebra:
$$
a \to F a
$$

Note that the catamorphism takes a value of `Fix F` and an algebra `Fa->a` and returns
a value of type `a`.

For Co-Algebras, the analogous of a catamorphism is an anamorphism. It takes a coalgebra
`a->Fa` and a value of type `a`, and returns a value of type `Fix F`.

```
ana :: Functor f => Coalgebra f a -> a -> Fix f
ana coa = In . fmap (ana coa) . coa
``` 

In [45]:
# Let coalg: A -> FA
ana(coalg::Function, x) = fix(fmap(y -> ana(coalg, y), coalg(x)))
ana(coalg::Function) = x -> ana(coalg,x) 

ana (generic function with 2 methods)

In [46]:
coalgSplitInt(n::Int) = @match n begin
    if n < 1 end  => NilF()
    n => ConsF(n,n-1)
end

coalgSplitInt(10)

ConsF{Int64, Int64}(10, 9)

In [47]:
ana(coalgSplitInt, 5)

Cons{Int64}(5, Cons{Int64}(4, Cons{Int64}(3, Cons{Int64}(2, Cons{Int64}(1, Nil())))))

### Hylomorphism

Let us now implement the factorial computation as a hylomorphism.
There are distinct ways to compute hylomorphisms. One is using anamorphisms
and catamorphisms:
```
    hylo(alg,coalg) = cata(alg) ∘ ana(coalg)
    hylo(alg,coalg,x) = cata(alg,ana(coalg,x))
```

The other way is using hylomorphism recursively:

In [48]:
hylo(alg, coalg, x) = alg(fmap(y->hylo(alg,coalg,y),coalg(x)));

Then we compute the factorial using our hylo. 

In [49]:
@show hylo(algProd,coalgSplitInt,4);

hylo(algProd, coalgSplitInt, 4) = 24


# Example 2

We start by defining the `NatF` functor, then the fixed-point `Nat`.
To visualize better the results, we implement the `tonat`, `toint` functions to turn our `Nat` values into integers.

In [53]:
@data NatF begin
    ZeroF()
    SuccF(_1)
end
fmap(f::Function, x::ZeroF)  = ZeroF()
fmap(f::Function, x::SuccF) = SuccF(f(x._1))

@data Nat begin
    Zero()
    Succ(Nat)
end
fmap(f::Function, x::Zero)  = Zero()
fmap(f::Function, x::Succ) = Succ(f(x._1))

fix(::ZeroF) = Zero()
fix(x::SuccF)= Succ(x._1)

unfix(::Zero) = ZeroF()
unfix(x::Succ)= SuccF(x._1)

toint(n::Nat)::Int = @match n begin 
    Zero() => 0
    Succ(x) => toint(x)+1
end
tonat(i::Int) = @match i begin
    if i < 0 end => Zero()
    0 => Zero()
    i => Succ(tonat(i-1))
end

@show toint(Zero())
@show toint(Succ(Zero()))
@show toint(Succ(Succ(Zero())))
@show tonat(10);

toint(Zero()) = 0
toint(Succ(Zero())) = 1
toint(Succ(Succ(Zero()))) = 2
tonat(10) = Succ(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Zero()))))))))))


### An F-Algebra and F-Coalgebra

We implement the `NatF`-algebras and coalgebra. We then use the hylomorphism.

In [51]:
algStr(x::ZeroF) = ""
algStr(x::SuccF) = x._1*"1"

coalgInt(i::Int) = @match i begin
    if i <= 0 end => ZeroF()
    i => SuccF(i-1)
end

hylo(algStr,coalgInt,10)

"1111111111"

### Computing Fibonacci using Hylomorphism

We implement the `fib` algebra, and use it in the hylomorphism 


In [57]:
fib(::ZeroF) = (1,1)
fib(x::SuccF) = (x._1[2], x._1[1] + x._1[1])

@show fib(SuccF((10,10)))
@show fib(ZeroF());

@show cata(fib, Succ(Succ(Zero())));

@show hylo(fib,coalgInt,10);

fib(SuccF((10, 10))) = (10, 20)
fib(ZeroF()) = (1, 1)
cata(fib, Succ(Succ(Zero()))) = (2, 2)
hylo(fib, coalgInt, 10) = (32, 32)


# Example 3 - QSort

In [58]:
@data TreeF{a} begin
    LeafF()
    NodeF(Int,::a, ::a)
end
fmap(f::Function, x::LeafF) = x
fmap(f::Function, x::NodeF) = NodeF(x._1,f(x._2),f(x._3))
NodeF(x::Int,y::TreeF{T},z::TreeF{T}) where T = NodeF{TreeF{T}}(x,y,z) 

@data Tree begin
    Leaf()
    Node(Int,::Tree, ::Tree)
end

fix(x::LeafF)  = Leaf()
fix(x::NodeF)= Node(x._1,x._2,x._3)
unfix(x::Leaf)  = LeafF{Any}()
unfix(x::Node)= NodeF(x._1,x._2,x._3)

NodeF(x::Int,y::Tree, z::Tree) = NodeF{Tree}(x::Int,y,z)

NodeF

In [59]:
"""
partition(f, x::Vector)
Given a list (vector) `x`, it partitions the list
in the first occurrence where `f` is satisfied.
"""
function partition(f, x::Vector{T}) where T
    index = map(f, x)
    x[index],x[.!(index)]
end

#   --------------|---------
x = [1,2,3,4,-12,0,5,10,0,1]
partition(x-> x ≤ 3,x)

([1, 2, 3, -12, 0, 0, 1], [4, 5, 10])

In [60]:
# TreeF-Coalgebra
# split : [Int] -> TreeF [Int]
function split(ns::Vector{Int})::TreeF{Vector{Int}}
    if isempty(ns)
        return LeafF{Vector{Int}}()
    end
    
    # Extract head and tail [1,2,3], after peel we get n = 1, ns = [2,3]
    # ns comes out as an iterator, hence we use `collect`.
    (n,ns) = Iterators.peel(ns)
    ns = collect(ns)
    
    (left, right) = partition(x -> x <= n, ns)
    return NodeF(n, left, right)
end

split (generic function with 1 method)

In [61]:
# TreeF-Algebra
# TreeF [Int] ->  [Int]
toList(x::LeafF) = Int[]
toList(x::NodeF{Vector{Int}}) = vcat(x._2,[x._1],x._3);

In [63]:
hylo(toList,split,[13,1,0,2,10,10])

6-element Vector{Int64}:
  0
  1
  2
 10
 10
 13

In [64]:
qsort = x::Vector{Int}->hylo(toList,split,x);

qsort([10,2,4,10])

4-element Vector{Int64}:
  2
  4
 10
 10